# Aeroacoustics

In this notebook we showcase how to set up and run an aeroacoustic simulation for a quadcopter using the Flow360 Python API. The process involves creating a project, defining multiple simulation runs by forking cases, and retrieving aeroacoustic results.

## 1. Setup and Imports

We begin by importing libraries and modules. `flow360` is the main package, and `pandas` is used for data handling of the results.

In [1]:
import pandas as pd

import flow360 as fl
from flow360.examples import Quadcopter

## 2. Project Creation

A Flow360 `Project` is a container for simulation assets such as geometries, meshes and cases. Here, the project is initialized from a pre-existing volume mesh file.

In [2]:
Quadcopter.get_files()

project = fl.Project.from_volume_mesh(
    Quadcopter.mesh_filename, name="Aeroacoustic results from Python"
)
vm = project.volume_mesh

[11:13:02] INFO: The file (quadcopter.cgns.zst) is being downloaded, please wait.

Output()

[11:13:20] INFO: VolumeMesh successfully submitted:                                                                
                   type   = Volume Mesh                                                                            
                   name   = Aeroacoustic results from Python                                                       
                   id     = vm-5b42dde6-4355-48e6-bfd1-210e6b980313                                                
                   status = uploaded                                                                               
           

Output()

## 3. Simulation Parameters

This section defines the simulation parameters. This includes defining the rotating zones, unsteady time-stepping parameters, and the physical models for the simulation.


### Rotation Zones

Provided volume mesh conatins four separate rotating zones, one for each rotor. These zones need to have their center of rotation and axis specified. The rotational speed `omega` is also defined.

In [3]:
with fl.SI_unit_system:
    rotation_zone_1 = vm["zone_r1"]
    rotation_zone_1.center = [-0.125, 0.125, 0.0055]
    rotation_zone_1.axis = [0, 0, 1]

    rotation_zone_2 = vm["zone_r2"]
    rotation_zone_2.center = [-0.125, -0.125, 0.0055]
    rotation_zone_2.axis = [0, 0, -1]

    rotation_zone_3 = vm["zone_r3"]
    rotation_zone_3.center = [0.125, -0.125, 0.0055]
    rotation_zone_3.axis = [0, 0, 1]

    rotation_zone_4 = vm["zone_r4"]
    rotation_zone_4.center = [0.125, 0.125, 0.0055]
    rotation_zone_4.axis = [0, 0, -1]

    omega = 6000 * fl.u.rpm

[11:13:56] INFO: using: SI unit system for unit inference.

### Time Stepping

For this unsteady simulation, the time step size and total number of steps are determined based on a desired rotational increment per step and the total number of rotor revolutions to be simulated. We will start with 3° of rotation per time step for 5 revolutions.

In [4]:
with fl.SI_unit_system:
    # Time step size is calculated based on a specified rotation per time step.
    deg_per_time_step_0 = 3.0 * fl.u.deg
    time_step_0 = deg_per_time_step_0 / omega.to("deg/s")

    # Total number of steps is determined by the required number of revolutions.
    revolution_time_0 = 360 * fl.u.deg / omega.to("deg/s")
    steps_0 = int(5 * revolution_time_0 / time_step_0)


           INFO: using: SI unit system for unit inference.

### Simulation Parameters Object

The `SimulationParams` object gathers all settings for a simulation run, including reference geometry dimensions, operating conditions, solver settings, and boundary conditions. Previously specified rotation zones will be assigned to a `Rotation` model.

In [5]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        reference_geometry=fl.ReferenceGeometry(
            area=0.0447726728530549,
            moment_center=[0, 0, 0],
            moment_length=[0.11938, 0.11938, 0.11938],
        ),
        operating_condition=fl.AerospaceCondition.from_mach(
            mach=0,
            thermal_state=fl.ThermalState(temperature=293.15),
            reference_mach=0.21868415800906676,
        ),
        time_stepping=fl.Unsteady(
            step_size=time_step_0,
            steps=steps_0,
        ),
        models=[
            fl.Fluid(
                navier_stokes_solver=fl.NavierStokesSolver(
                    absolute_tolerance=1e-10,
                    relative_tolerance=0.1,
                    order_of_accuracy=1,
                    linear_solver=fl.LinearSolver(max_iterations=30),
                ),
                turbulence_model_solver=fl.SpalartAllmaras(
                    absolute_tolerance=1e-8,
                    relative_tolerance=0.1,
                    order_of_accuracy=1,
                    linear_solver=fl.LinearSolver(max_iterations=20),
                ),
            ),
            fl.Wall(
                surfaces=[
                    vm["zone_s/airframe"],
                    vm["zone_r1/blade1"],
                    vm["zone_r2/blade2"],
                    vm["zone_r3/blade3"],
                    vm["zone_r4/blade4"],
                ],
            ),
            fl.Freestream(surfaces=vm["zone_s/farfield"]),
            fl.Rotation(
                name="Rotation",
                volumes=[rotation_zone_1, rotation_zone_2, rotation_zone_3, rotation_zone_4],
                spec=fl.AngularVelocity(value=omega),
            ),
        ],
    )

[11:13:57] INFO: using: SI unit system for unit inference.

## 4. Executing the Simulation and Forking

The simulation is executed in stages. As can be seen based on the settings up above, our first run will be a coarse, first-order simulation in order to establish an initial flow field. Subsequent simulations are then 'forked' from previous ones. Forking a case initializes a new simulation from the results of an existing one, which can significantly reduce computational cost when iterating on simulation parameters. This is particularly useful for changing numerical schemes or introducing new outputs without restarting the simulation from scratch.

> Note: First order run is not required here, but is used for demonstration purposes nonetheless.

In [6]:
case = project.run_case(params, "First order run")

           INFO: using: SI unit system for unit inference.

[11:13:59] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = First order run                                                                        
                   id     = case-cefd971c-0ce1-438d-afd8-5dbf4bc5d6a6                                              
                   status = pending                                                                                
           

### Second Run with Higher Order Accuracy

For the second run, the order of accuracy for the solvers is increased to 2, and the time step is refined. The case is forked from the initial run to leverage the already computed flow field.

In [7]:
# For the second run, parameters are updated directly on the params object of the completed case.
# The order of accuracy is increased and the time step size is refined.
case.params.models[0].navier_stokes_solver.order_of_accuracy = 2
case.params.models[0].navier_stokes_solver.linear_solver = fl.LinearSolver(max_iterations=25)

case.params.models[0].turbulence_model_solver.order_of_accuracy = 2
case.params.models[0].turbulence_model_solver.linear_solver = fl.LinearSolver(max_iterations=25)

with fl.SI_unit_system:
    deg_per_time_step_1 = 0.404496 * fl.u.deg
    time_step_1 = deg_per_time_step_1 / omega.to("deg/s")

    revolution_time_1 = 360 * fl.u.deg / omega.to("deg/s")
    steps_1 = int(5 * revolution_time_1 / time_step_1)

    case.params.time_stepping.step_size = time_step_1
    case.params.time_stepping.steps = steps_1

case_fork_1 = project.run_case(case.params, "Second order run", fork_from=case)

Output()

[11:14:01] INFO: using: SI unit system for unit inference.

           INFO: using: SI unit system for unit inference.

           INFO: using: SI unit system for unit inference.

[11:14:03] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Second order run                                                                       
                   id     = case-6a7312f0-be84-4e3c-a879-0bf7d68667a3                                              
                   status = pending                                                                                
           

### Final Run with Aeroacoustic Outputs

The final simulation is forked from the second-order run. `AeroAcousticOutput` is added to the simulation parameters to collect acoustic data from specified observer points.

In [8]:
case_fork_1.params.outputs = [
    fl.AeroAcousticOutput(
        observers=[
            fl.Observer(position=[0, -1.905, 0] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -1.7599905, -0.72901194] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -1.3470384, -1.3470384] * fl.u.m, group_name="1"),
            fl.Observer(position=[0.9525, -0.9525, -1.3470384] * fl.u.m, group_name="1"),
            fl.Observer(position=[1.3470384, 0, -1.3470384] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, 0, 1.905] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -0.37164706, 1.8683959] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -1.868396, 0.37164707] * fl.u.m, group_name="1"),
            fl.Observer(position=[1.295, 0, -0.767] * fl.u.m, group_name="2"),
        ],
        write_per_surface_output=True,
    )
]

case_fork_2 = project.run_case(case_fork_1.params, "Final run", fork_from=case_fork_1)

Output()

[11:14:05] INFO: using: SI unit system for unit inference.

           INFO: using: SI unit system for unit inference.

[11:14:07] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Final run                                                                              
                   id     = case-4342754a-91b3-48c7-9fac-d63804742bdf                                              
                   status = pending                                                                                
           

## 5. Retrieving Results

After launching the simulations, the script waits for the final case to complete. The results, including the aeroacoustic data, can then be accessed and downloaded for analysis.

In [ ]:
case_fork_2.wait()

Output()

In [ ]:
results = case_fork_2.results

total_acoustics = results.aeroacoustics
print(total_acoustics)

### Surface-Specific Outputs

In addition to the total aeroacoustic output from all surfaces, individual surface contributions can be downloaded. The following cell downloads the acoustic data for each of the four rotors and loads them into pandas DataFrames.

In [ ]:
# There are also surface specific aeroacoustic output files
blade_1_acoustics = results.download_file_by_name(
    "results/surface_zone_r1_blade1_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_1_acoustics = pd.read_csv(blade_1_acoustics)
print("Blade 1 acoustics:")
print(blade_1_acoustics)

blade_2_acoustics = results.download_file_by_name(
    "results/surface_zone_r2_blade2_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_2_acoustics = pd.read_csv(blade_2_acoustics)
print("\nBlade 2 acoustics:")
print(blade_2_acoustics)

blade_3_acoustics = results.download_file_by_name(
    "results/surface_zone_r3_blade3_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_3_acoustics = pd.read_csv(blade_3_acoustics)
print("\nBlade 3 acoustics:")
print(blade_3_acoustics)

blade_4_acoustics = results.download_file_by_name(
    "results/surface_zone_r4_blade4_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_4_acoustics = pd.read_csv(blade_4_acoustics)
print("\nBlade 4 acoustics:")
print(blade_4_acoustics)